In [2]:
import warnings
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
Train = pd.read_parquet("train_temporal_classification.parquet.gz")
Test =  pd.read_parquet("test_temporal_classification.parquet.gz")

In [4]:
Train.head()

,powiat_voivod,cloud_cover,wind_speed,humidity,precipitation,snow_depth,temperature,voivodship,animal stock - total,area by land - utilised agricultural area,...,season,CAQI_lag1,holiday_name,is_school_holiday,is_lockdown,is_have_coalplant,rolling7_CAQI,rolling30_CAQI,rolling90_CAQI,is_weekend
DATE,,,,,,,,,,,,,,,,,,,,,
2017-01-01,"powiat aleksandrowski, kujawsko-pomorskie",4.746750,0.0,88.4,1.0,0.0,1.8,kujawsko-pomorskie,44.997895,1.718015,...,Winter,24.968064,New Year's Day,0.0,0.0,0.0,24.968064,28.884791,47.077522,1
2017-01-01,"powiat augustowski, podlaskie",8.000000,50.0,95.0,9.0,0.0,0.4,podlaskie,47.547318,4.963793,...,Winter,61.553125,New Year's Day,0.0,0.0,0.0,61.553125,36.561883,54.128302,1
2017-01-01,"powiat bełchatowski, łódzkie",5.923833,0.0,80.0,0.0,5.0,0.0,łódzkie,43.636364,3.809493,...,Winter,11.000000,New Year's Day,0.0,0.0,1.0,11.000000,29.314815,48.801235,1
2017-01-01,"powiat biała podlaska, lubelskie",6.000000,0.0,78.0,0.0,0.0,0.9,lubelskie,21.265306,0.138255,...,Winter,71.000000,New Year's Day,0.0,0.0,0.0,71.000000,59.414286,70.084646,1
2017-01-01,"powiat białystok, podlaskie",8.000000,0.0,89.0,0.0,0.0,0.4,podlaskie,47.549020,0.305201,...,Winter,49.878097,New Year's Day,0.0,0.0,0.0,49.878097,33.337733,50.529500,1


In [7]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 300960 entries, 2017-01-01 to 2021-02-28
Columns: 131 entries, powiat_voivod to is_weekend
dtypes: float64(120), int32(1), int64(5), object(5)
memory usage: 301.9+ MB


In [8]:
Train.isnull().sum()

powiat_voivod        0
cloud_cover          0
wind_speed           0
humidity             0
precipitation        0
                    ..
is_have_coalplant    0
rolling7_CAQI        0
rolling30_CAQI       0
rolling90_CAQI       0
is_weekend           0
Length: 131, dtype: int64

In [9]:
Train.shape

(300960, 131)

In [10]:
Train.columns

Index(['powiat_voivod', 'cloud_cover', 'wind_speed', 'humidity',
       'precipitation', 'snow_depth', 'temperature', 'voivodship',
       'animal stock - total', 'area by land - utilised agricultural area',
       ...
       'season', 'CAQI_lag1', 'holiday_name', 'is_school_holiday',
       'is_lockdown', 'is_have_coalplant', 'rolling7_CAQI', 'rolling30_CAQI',
       'rolling90_CAQI', 'is_weekend'],
      dtype='object', length=131)

In [12]:
# combine high and vhigh to a single category, as we want to see how it will affect the accuracy of model
Train['CAQI_level'] = Train['CAQI_level'].replace({'vhigh':'high'})

In [13]:
Train['CAQI_level'].value_counts()

vlow      134555
low       126762
medium     30737
high        8906
Name: CAQI_level, dtype: int64

In [14]:
# The ordered of mapping
mapping = {'vlow': 0, 'low': 1, 'medium': 2, 'high': 3}
Train['CAQI_level'] = Train['CAQI_level'].replace(mapping)

In [15]:
Train['CAQI_level'].value_counts()

0    134555
1    126762
2     30737
3      8906
Name: CAQI_level, dtype: int64

In [7]:
Test.columns.to_list()

['powiat_voivod',
 'cloud_cover',
 'wind_speed',
 'humidity',
 'precipitation',
 'snow_depth',
 'temperature',
 'voivodship',
 'animal stock - total',
 'area by land - utilised agricultural area',
 'area by land - forests',
 'area by land - residential areas',
 'area by land - industrial lands',
 'area by land - recreational and rest areas',
 'area by land - lands under waters',
 'area by land - wasteland',
 'area by land - other area',
 'emission of particulates - fugitive',
 'emission of particulates - fuel combustion',
 'emission of particulates - cement/lime and refractory materials',
 'emission of particulates - silicon',
 'emission of particulates - chemical fertilizers',
 'emission of particulates - surface-active agents',
 'emission of particulates - carbon and graphite, soot',
 'emission of pollutant gases - fugitive',
 'emission of pollutant gases - sulphur dioxide',
 'emission of pollutant gases - nitrogen oxides',
 'emission of pollutant gases - carbon monoxide',
 'emission

In [8]:
# # encoding the target variable
# le = LabelEncoder()
# Train['CAQI_level_encoded'] = le.fit_transform(Train['CAQI_level'])

In [9]:
# print('Label encoding mapping:')
# for i, cat in enumerate(le.classes_):
#     print(f'{cat} -> {i}')

Label encoding mapping:
high -> 0
low -> 1
medium -> 2
vhigh -> 3
vlow -> 4


In [6]:
Train.columns.tolist()

['powiat_voivod',
 'cloud_cover',
 'wind_speed',
 'humidity',
 'precipitation',
 'snow_depth',
 'temperature',
 'voivodship',
 'animal stock - total',
 'area by land - utilised agricultural area',
 'area by land - forests',
 'area by land - residential areas',
 'area by land - industrial lands',
 'area by land - recreational and rest areas',
 'area by land - lands under waters',
 'area by land - wasteland',
 'area by land - other area',
 'emission of particulates - fugitive',
 'emission of particulates - fuel combustion',
 'emission of particulates - cement/lime and refractory materials',
 'emission of particulates - silicon',
 'emission of particulates - chemical fertilizers',
 'emission of particulates - surface-active agents',
 'emission of particulates - carbon and graphite, soot',
 'emission of pollutant gases - fugitive',
 'emission of pollutant gases - sulphur dioxide',
 'emission of pollutant gases - nitrogen oxides',
 'emission of pollutant gases - carbon monoxide',
 'emission

In [16]:
from sklearn.preprocessing import LabelEncoder
# encoding the other categorical columns
cat_cols = ['powiat_voivod', 'voivodship', 'season','holiday_name']
for col in cat_cols:
    le = LabelEncoder()
    Train['powiat_voivod_encoded'] = le.fit_transform(Train['powiat_voivod'])
    Train['voivodship_encoded'] = le.fit_transform(Train['voivodship'])
    Train['season_encoded'] = le.fit_transform(Train['season'])
    Train['holiday_name_encoded'] = le.fit_transform(Train['holiday_name'])

# TEST DATA

In [17]:
Test.head()

,powiat_voivod,cloud_cover,wind_speed,humidity,precipitation,snow_depth,temperature,voivodship,animal stock - total,area by land - utilised agricultural area,...,season,CAQI_lag1,holiday_name,is_school_holiday,is_lockdown,is_have_coalplant,rolling7_CAQI,rolling30_CAQI,rolling90_CAQI,is_weekend
DATE,,,,,,,,,,,,,,,,,,,,,
2021-03-02,"powiat aleksandrowski, kujawsko-pomorskie",5.000000,0.0,92.5,0.0,0.0,2.7,kujawsko-pomorskie,39.393684,1.712729,...,Spring,23.821706,Non Holiday,0.0,0.0,0.0,41.629324,44.641164,33.798813,0
2021-03-02,"powiat augustowski, podlaskie",7.000000,60.0,90.0,0.0,0.0,2.3,podlaskie,51.660036,4.947356,...,Spring,20.593006,Non Holiday,0.0,0.0,0.0,51.103275,39.976399,33.184952,0
2021-03-02,"powiat bełchatowski, łódzkie",4.846046,0.0,85.0,0.0,0.0,3.7,łódzkie,42.485537,3.735109,...,Spring,21.945455,Non Holiday,0.0,0.0,1.0,42.552922,40.208157,33.076156,0
2021-03-02,"powiat biała podlaska, lubelskie",6.000000,0.0,89.0,0.0,0.0,3.7,lubelskie,19.326531,0.136890,...,Spring,33.166667,Non Holiday,0.0,0.0,0.0,58.112608,57.730682,48.761292,0
2021-03-02,"powiat białystok, podlaskie",7.000000,0.0,91.5,0.0,0.0,2.5,podlaskie,51.666667,0.304191,...,Spring,21.850145,Non Holiday,0.0,0.0,0.0,50.750194,46.867564,38.484418,0


In [9]:
Test.columns.to_list()

['powiat_voivod',
 'cloud_cover',
 'wind_speed',
 'humidity',
 'precipitation',
 'snow_depth',
 'temperature',
 'voivodship',
 'animal stock - total',
 'area by land - utilised agricultural area',
 'area by land - forests',
 'area by land - residential areas',
 'area by land - industrial lands',
 'area by land - recreational and rest areas',
 'area by land - lands under waters',
 'area by land - wasteland',
 'area by land - other area',
 'emission of particulates - fugitive',
 'emission of particulates - fuel combustion',
 'emission of particulates - cement/lime and refractory materials',
 'emission of particulates - silicon',
 'emission of particulates - chemical fertilizers',
 'emission of particulates - surface-active agents',
 'emission of particulates - carbon and graphite, soot',
 'emission of pollutant gases - fugitive',
 'emission of pollutant gases - sulphur dioxide',
 'emission of pollutant gases - nitrogen oxides',
 'emission of pollutant gases - carbon monoxide',
 'emission

In [11]:
# encoding the target variable
# le = LabelEncoder()
# Test['CAQI_level_encoded'] = le.fit_transform(Test['CAQI_level'])

In [12]:
# print('Label encoding mapping:')
# for i, cat in enumerate(le.classes_):
#     print(f'{cat} -> {i}')

Label encoding mapping:
high -> 0
low -> 1
medium -> 2
vhigh -> 3
vlow -> 4


In [13]:
# Test = Test.drop(['CAQI_level'], axis=1)

In [14]:
# # Dropping the holiday column as well, as it throws an error when encoding
# Test = Test.drop(['holiday_name'], axis=1)

In [18]:
# combine high and vhigh to a single category, as we want to see how it will affect the accuracy of model
Test['CAQI_level'] = Test['CAQI_level'].replace({'vhigh':'high'})

In [19]:
Test['CAQI_level'].value_counts()

vlow      31913
low       24346
medium     3683
high        448
Name: CAQI_level, dtype: int64

In [20]:
# order of mapping
mapping = {'vlow':0,'low':1,"medium":2,"high":3}
Test['CAQI_level'] = Test['CAQI_level'].replace(mapping)

In [21]:
Test['CAQI_level'].value_counts()

0    31913
1    24346
2     3683
3      448
Name: CAQI_level, dtype: int64

In [22]:
from sklearn.preprocessing import LabelEncoder
# encoding the other categorical columns
cat_cols = ['powiat_voivod', 'voivodship', 'season','holiday_name']
for col in cat_cols:
    le = LabelEncoder()
    Test['powiat_voivod_encoded'] = le.fit_transform(Test['powiat_voivod'])
    Test['voivodship_encoded'] = le.fit_transform(Test['voivodship'])
    Test['season_encoded'] = le.fit_transform(Test['season'])
    Test['holiday_name_encoded'] = le.fit_transform(Test['holiday_name'])

In [23]:
X_train = Train.drop(['CAQI_level','powiat_voivod', 'voivodship', 'season', 'holiday_name'], axis=1)
y_train = Train['CAQI_level']
X_test = Test.drop(['CAQI_level','powiat_voivod', 'voivodship', 'season', 'holiday_name'], axis=1)
y_test = Test['CAQI_level']

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [26]:
# Predict on the test set
y_pred = clf.predict(X_test)

# Accuracy score:
model_accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", model_accuracy)

Model Accuracy: 0.690528233151184


Model Accuracy:(69.09%) 

In [16]:
from sklearn.metrics import confusion_matrix

In [27]:
# Evaluating the model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.81      0.78     31913
           1       0.62      0.64      0.63     24346
           2       0.00      0.00      0.00      3683
           3       0.00      0.00      0.00       448

    accuracy                           0.69     60390
   macro avg       0.34      0.36      0.35     60390
weighted avg       0.64      0.69      0.67     60390



In [28]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)

print(cm)

[[26003  5910     0     0]
 [ 8648 15698     0     0]
 [  366  3317     0     0]
 [    8   440     0     0]]


In [29]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

#Defining the k-fold cross-validation object
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Computing the cross-validation scores
# Used the training set data only for cross validation.
cv_scores = cross_val_score(clf, X_train, y_train, cv=kfold, scoring='accuracy')

# Cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

Cross-validation scores: [0.65530303 0.65201356 0.64804293 0.65257842 0.64661417]
Mean CV accuracy: 0.6509104199893674
